Tutorial - MultiRotor System
======================

This is a basic tutorial on how to use the `MultiRotor` class for rotordynamics analysis. Before starting this tutorial, be sure you're already familiar with ROSS library. What changes here is basically the part of building the model, since running the analyses will be practically the same as seen in the other tutorials.

When two shafts are joined together by gears, coupling can occur between the lateral and torsional vibration. This interaction is modeled in ROSS based on Rao et al. (1998). In this work, a typical spur gear pair is modeled as a pair of rigid disks connected by a spring and a damping, considering the pressure angle ($\alpha$) and the oritentation angle ($\varphi$) as shown in the following:

<div style="text-align: center;">
    <img src="../_static/img/img_tutorial4_gearmesh.png" alt="Gear mesh" style="width: 450px; height: auto;">
    <br>
    <small>Figure 1: Global coordinate system of a spur gear pair (Yang et al., 2016).</small>
</div>

As you can see, an element not yet shown is needed to model the multi-rotor system, the `GearElement`. This new element resembles the `DiskElement` with added attributes, which will be shown next.


## Section 1: GearElement Class

The class `GearElement` allows you to create gear elements. It is a subclass of `DiskElement` that requires information related to its pitch diameter and pressure angle.

ROSS offers 2 (two) ways to create a gear element:
1. Inputing mass and inertia data
2. Inputing geometrical and material data

### 1.1 Creating a single gear element

In this tutorial, only an example of how to create a single element will be shown. However, the same procedures presented for the `DiskElement` in the first part of the <b>Tutorial</b> can be replicated here. The `GearElement` goes into the same list of disk elements when assembling the rotor.

This example below shows how to instantiate a gear element according to the mass and inertia properties:

In [1]:
import sys

sys.path.append("/home/jguarato/Documents/GitHub/ROSS_Code/ross")
sys.path.append("/home/jguarato/Documents/GitHub/ROSS_Code/")

import ross as rs
import numpy as np
import pandas as pd

import pickle
import plotly.graph_objects as go

from ross.units import Q_

In [ ]:
gear = rs.GearElement(
    n=0,
    m=5,
    Id=0.002,
    Ip=0.004,
    pitch_diameter=0.5,
    pressure_angle=Q_(22.5, "deg"),
    tag="Gear",
)
gear

## Section 2: MultiRotor Class

`MultiRotor` is a subclass of `Rotor` class. It takes two rotors (driving and driven) as arguments and couple them with their gears. The object created has several methods that can be used to evaluate the dynamics of the model (they all start with the prefix `.run_`).

To use this class, you must input the already instantiated rotors and each one need at least one gear element.

The shaft elements are renumbered starting with the elements of the driving rotor.

To assemble the matrices, the driving and driven matrices are joined.
For the stiffness matrix, the coupling is considered at the nodes of the gears in contact.

### 2.1 Creating a multi-rotor model
Let's create a simple model with two rotors connected by a pair of spur gears and supported by flexible bearings, as shown in Fig. 2. For more details on the description of the model, see the work of Rao et al. (1998).

<div style="text-align: center;">
    <img src="../_static/img/img_tutorial4_multirotor.png" alt="MultiRotor" style="width: 450px; height: auto;">
    <br>
    <small>Figure 2: Rotors connected by a pair of spur gears (Friswell et al., 2010).</small>
</div>

In Figure 2, the first node is 1 (one), but we must remember that in ROSS the node count starts at 0 (zero).

#### 2.1.1 Creating material

In [3]:
# Creating material
material = rs.Material(name="mat_steel", rho=7800, E=207e9, G_s=79.5e9)

#### 2.1.2 Creating the driving rotor

In [ ]:
# Creating rotor 1
L1 = [0.1, 4.24, 1.16, 0.3]
d1 = [0.3, 0.3, 0.22, 0.22]
shaft1 = [
    rs.ShaftElement(
        L=L1[i],
        idl=0.0,
        odl=d1[i],
        material=material,
        shear_effects=True,
        rotary_inertia=True,
        gyroscopic=True,
    )
    for i in range(len(L1))
]

generator = rs.DiskElement(
    n=1,
    m=525.7,
    Id=16.1,
    Ip=32.2,
)
disk = rs.DiskElement(
    n=2,
    m=116.04,
    Id=3.115,
    Ip=6.23,
)

pressure_angle = rs.Q_(22.5, "deg").to_base_units().m
base_radius = 0.5086
pitch_diameter = 2 * base_radius / np.cos(pressure_angle)
gear1 = rs.GearElement(
    n=4,
    m=726.4,
    Id=56.95,
    Ip=113.9,
    pitch_diameter=pitch_diameter,
    pressure_angle=pressure_angle,
)

bearing1 = rs.BearingElement(n=0, kxx=183.9e6, kyy=200.4e6, cxx=3e3)
bearing2 = rs.BearingElement(n=3, kxx=183.9e6, kyy=200.4e6, cxx=3e3)

rotor1 = rs.Rotor(
    shaft1,
    [generator, disk, gear1],
    [bearing1, bearing2],
)

rotor1.plot_rotor()

#### 2.1.3 Creating the driven rotor

In [ ]:
# Creating rotor 2

L2 = [0.3, 5, 0.1]
d2 = [0.15, 0.15, 0.15]
shaft2 = [
    rs.ShaftElement(
        L=L2[i],
        idl=0.0,
        odl=d2[i],
        material=material,
        shear_effects=True,
        rotary_inertia=True,
        gyroscopic=True,
    )
    for i in range(len(L2))
]

base_radius = 0.03567
pitch_diameter = 2 * base_radius / np.cos(pressure_angle)
gear2 = rs.GearElement(
    n=0,
    m=5,
    Id=0.002,
    Ip=0.004,
    pitch_diameter=pitch_diameter,
    pressure_angle=pressure_angle,
)
turbine = rs.DiskElement(n=2, m=7.45, Id=0.0745, Ip=0.149)

bearing3 = rs.BearingElement(n=1, kxx=10.1e6, kyy=41.6e6, cxx=3e3)
bearing4 = rs.BearingElement(n=3, kxx=10.1e6, kyy=41.6e6, cxx=3e3)

rotor2 = rs.Rotor(
    shaft2,
    [gear2, turbine],
    [bearing3, bearing4],
)

rotor2.plot_rotor()

#### 2.1.4 Connecting rotors

To build the multi-rotor model, we need to inform, in the following order:
- the driving rotor,
- the driven rotor,
- the tuple with the pair of coupled nodes (first number corresponds to the gear node of the driving rotor, and the second of the driven rotor),
- the gear ratio, and
- the gear mesh stiffness.

Finally, we can inform:
- the orientation angle (if not defined, zero is adopted as the default),
- the position of the driven rotor in relation to the driving rotor only for visualization in the plot ("above" or "below"), and
- a tag.

In [ ]:
# Creating multi-rotor

N1 = 328  # Number of teeth of gear 1
N2 = 23  # Number of teeth of gear 2
k_mesh = 1e8  # Mesh stiffness
gear_ratio = N1 / N2

multirotor = rs.MultiRotor(
    rotor1,
    rotor2,
    coupled_nodes=(4, 0),
    gear_ratio=gear_ratio,
    gear_mesh_stiffness=k_mesh,
    orientation_angle=0,
    position="below",
)

multirotor.plot_rotor()

### 2.2 Running analyses
We will run some analyses for the multi-rotor in this section and even compare results from the literature.

#### 2.2.1 Modal analysis

Let's start with the modal analysis to obtain the natural frequencies for the coupled rotor when the generator runs at
1500 RPM. Then we will compare the results with Friswell et al. (2010).

It is worth noting that in the analyses, we must always inform the respective speed of the driving rotor and not the driven one.


In [ ]:
# Friswell et al. (2010) results for natural frequencies:
Friswell_results = np.array(
    [
        11.641,
        12.284,
        17.268,
        18.458,
        23.956,
        37.681,
        49.889,
        50.861,
        56.248,
        57.752,
        59.188,
        63.113,
        74.203,
    ]
)

speed = rs.Q_(1500, "RPM")
frequencies = 13

modal = multirotor.run_modal(speed, num_modes=2 * frequencies)
wn = np.round(rs.Q_(modal.wn, "rad/s").to("Hz").m, 3)

print("Natural frequencies (Hz)")
pd.DataFrame(
    {
        "Friswell et al.": Friswell_results,
        "ROSS": wn,
        "Error (%)": np.abs(wn - Friswell_results) / wn * 100,
    }
)

### 2.2.2 Campbell diagram

To obtain the Campbell diagram we can proceed in the same way as seen for a single rotor. Remember that the reference speeds / frequencies are in relation to the driving rotor.

In the Campbell diagram below, the dashed lines show the shaft rotation speeds corresponding to the generator (blue, node 1) and turbine (yellow, node 7).

In [ ]:
frequency_range = rs.Q_(np.arange(0, 5000, 100), "RPM")

campbell = multirotor.run_campbell(frequency_range, frequencies=13)
campbell.plot(frequency_units="Hz", harmonics=[1, round(gear_ratio, 3)]).show()

### 2.2.3 Unbalance response



In [17]:
nodes = [2, 7]
unb_mag = [35.505e-3, 0.449e-3]
unb_phase = [0, 0]

dt = 0.1e-4
t = np.arange(0, 900, dt)
speed1 = rs.Q_(5000, "RPM").to_base_units().m  # Generator rotor speed

num_dof = multirotor.number_dof

In [ ]:
# Unbalance force
F = np.zeros((len(t), multirotor.ndof))

for i, node in enumerate(nodes):
    speed = multirotor._check_speed(node, speed1)
    phi = speed * t + unb_phase[i]

    dofx = num_dof * node + 0
    dofy = num_dof * node + 1
    F[:, dofx] += unb_mag[i] * (speed**2) * np.cos(phi)
    F[:, dofy] += unb_mag[i] * (speed**2) * np.sin(phi)

In [15]:
# Time response
time_resp = multirotor.run_time_response(speed1, F, t)
amp_resp = time_resp.yout

In [ ]:
# compare results
Yang_results = np.array(
    [7.42285, 6.92465, 2.46929, 1.42028, 1.45093, 2.32250, 1.30708, 1.62738, 1.90581]
)

max_resp = []
for node in range(9):
    init_step = int(2 * len(t) / 3)

    dofx = num_dof * node + 0
    dofy = num_dof * node + 1
    x = amp_resp[init_step:, dofx]
    y = amp_resp[init_step:, dofy]

    max_resp.append(max(np.sqrt(x**2 + y**2)) / 1e-5)

print("Maximum unbalance responses (m)")
pd.DataFrame(
    {
        "Yang et al.": Yang_results,
        "ROSS": max_resp,
        "Error (%)": np.abs(max_resp - Yang_results) / Yang_results * 100,
    }
)

----
<b>References</b>

Friswell, M. I. (2010). Dynamics of rotating machines. Cambridge university press.

Rao, J.S., Shiau, T.N. and Chang, J.R. (1998). Theoretical analysis of lateral response due to torsional excitation of geared rotors. <i>Mechanism and Machine Theory</i>, 33 (6), 761-783.

Yang, Y., Wang, J., Wang, X. and Dai, Y. (2016). A general method to predict unbalance responses of geared rotor systems. <i>Journal of Sound and Vibration</i>, 381, 246-263.